# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD

## 1.Load Data

In [2]:
data= pd.read_csv('CarPrice_Assignment.csv')

## 2.Check Missing Values

In [3]:
data.isnull().values.any()

False

In [4]:
len(data)

205

In [5]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


## convert non-numerical values into numerical values::

In [6]:
d= data.dtypes == 'object'

In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

for i in range(len(data.columns)):
    if (d[i] == True and ((data.columns[i]!='CarName') and (data.columns[i]!='doornumber'))):
        data.iloc[:, i] = labelencoder.fit_transform(data.iloc[:, i])
    elif (data.columns[i]=='doornumber') :
        data['doornumber'] = data['doornumber'].map({'two':2,'four':4})

In [8]:
data.head(4)

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,1,0,2,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,1,0,2,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,1,0,2,2,2,0,94.5,...,152,5,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,1,0,4,3,1,0,99.8,...,109,5,3.19,3.40,10.0,102,5500,24,30,13950.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    int32  
 4   aspiration        205 non-null    int32  
 5   doornumber        205 non-null    int64  
 6   carbody           205 non-null    int32  
 7   drivewheel        205 non-null    int32  
 8   enginelocation    205 non-null    int32  
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    int32  
 15  cylindernumber    205 non-null    int32  
 16  enginesize        205 non-null    int64  
 1

In [10]:
corr = data.corr()

In [11]:
data

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,1,0,2,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,1,0,2,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,1,0,2,2,2,0,94.5,...,152,5,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,1,0,4,3,1,0,99.8,...,109,5,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,1,0,4,3,0,0,99.4,...,136,5,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),1,0,4,3,2,0,109.1,...,141,5,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,1,1,4,3,2,0,109.1,...,141,5,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,1,0,4,3,2,0,109.1,...,173,5,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,0,1,4,3,2,0,109.1,...,145,3,3.01,3.40,23.0,106,4800,26,27,22470.0


In [12]:
corr.where(corr>=0.4, inplace=True)

In [13]:
data1= corr.iloc[24:,:].dropna(axis=1)

In [14]:
data2=data[data1.columns]

In [15]:
data2.head()

,drivewheel,wheelbase,carlength,carwidth,curbweight,enginesize,fuelsystem,boreratio,horsepower,price
0,2,88.6,168.8,64.1,2548,130,5,3.47,111,13495.0
1,2,88.6,168.8,64.1,2548,130,5,3.47,111,16500.0
2,2,94.5,171.2,65.5,2823,152,5,2.68,154,16500.0
3,1,99.8,176.6,66.2,2337,109,5,3.19,102,13950.0
4,0,99.4,176.6,66.4,2824,136,5,3.19,115,17450.0


### 3.Split Data

In [16]:
x = (data2.loc[:, data2.columns != 'price'])
y = (data2.loc[:, data2.columns == 'price'])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [17]:
x_train.shape

(143, 9)

### 4.Model

In [18]:
from keras import models
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',input_shape=(9,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

    return model

In [19]:
k = 4
num_val_samples = len(x_train) // k

In [21]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partialx_train = np.concatenate(
    [x_train[:i * num_val_samples],
    x_train[(i + 1) * num_val_samples:]],
    axis=0)
    partialy_train = np.concatenate(
    [y_train[:i * num_val_samples],
    y_train[(i + 1) * num_val_samples:]],
    axis=0)
    model = build_model()
    history = model.fit(partialx_train, partialy_train,
    validation_data=(val_data, val_targets),
    epochs=num_epochs, batch_size=32, verbose=1)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

processing fold # 0
Epoch 1/500
4/4 [==============================] - 0s 82ms/step - loss: 252058944.0000 - mae: 13575.7773 - val_loss: 175856080.0000 - val_mae: 11813.1035
Epoch 2/500
4/4 [==============================] - 0s 14ms/step - loss: 238928560.0000 - mae: 13137.5566 - val_loss: 168047296.0000 - val_mae: 11498.9023
Epoch 3/500
4/4 [==============================] - 0s 12ms/step - loss: 229816096.0000 - mae: 12816.1133 - val_loss: 161090448.0000 - val_mae: 11211.5205
Epoch 4/500
4/4 [==============================] - 0s 16ms/step - loss: 221494208.0000 - mae: 12516.7520 - val_loss: 154663456.0000 - val_mae: 10939.7627
Epoch 5/500
4/4 [==============================] - 0s 14ms/step - loss: 213427136.0000 - mae: 12230.3496 - val_loss: 147886784.0000 - val_mae: 10645.9531
Epoch 6/500
4/4 [==============================] - 0s 14ms/step - loss: 204950512.0000 - mae: 11918.6523 - val_loss: 140904016.0000 - val_mae: 10334.6895
Epoch 7/500
4/4 [==============================] - 0s 11

4/4 [==============================] - 0s 11ms/step - loss: 33360538.0000 - mae: 4435.7959 - val_loss: 21278710.0000 - val_mae: 3464.5251
Epoch 109/500
4/4 [==============================] - 0s 14ms/step - loss: 33880456.0000 - mae: 4347.7183 - val_loss: 21723400.0000 - val_mae: 3547.1794
Epoch 110/500
4/4 [==============================] - 0s 13ms/step - loss: 33126580.0000 - mae: 4463.2163 - val_loss: 21406334.0000 - val_mae: 3491.1221
Epoch 111/500
4/4 [==============================] - 0s 12ms/step - loss: 33247602.0000 - mae: 4408.4980 - val_loss: 21279324.0000 - val_mae: 3468.8816
Epoch 112/500
4/4 [==============================] - 0s 12ms/step - loss: 33543132.0000 - mae: 4413.4199 - val_loss: 21959250.0000 - val_mae: 3592.6074
Epoch 113/500
4/4 [==============================] - 0s 12ms/step - loss: 33053094.0000 - mae: 4484.8745 - val_loss: 21727336.0000 - val_mae: 3553.7285
Epoch 114/500
4/4 [==============================] - 0s 14ms/step - loss: 33113654.0000 - mae: 4459.32

Epoch 162/500
4/4 [==============================] - 0s 14ms/step - loss: 31832922.0000 - mae: 4366.8955 - val_loss: 20080670.0000 - val_mae: 3333.7395
Epoch 163/500
4/4 [==============================] - 0s 12ms/step - loss: 31637342.0000 - mae: 4271.2090 - val_loss: 20212934.0000 - val_mae: 3359.7852
Epoch 164/500
4/4 [==============================] - 0s 10ms/step - loss: 31785552.0000 - mae: 4320.7598 - val_loss: 20360076.0000 - val_mae: 3389.5020
Epoch 165/500
4/4 [==============================] - 0s 12ms/step - loss: 31547276.0000 - mae: 4311.6113 - val_loss: 20779616.0000 - val_mae: 3471.4172
Epoch 166/500
4/4 [==============================] - 0s 12ms/step - loss: 31565828.0000 - mae: 4404.3994 - val_loss: 20472288.0000 - val_mae: 3415.2356
Epoch 167/500
4/4 [==============================] - 0s 12ms/step - loss: 31640652.0000 - mae: 4373.2515 - val_loss: 20479982.0000 - val_mae: 3419.3357
Epoch 168/500
4/4 [==============================] - 0s 10ms/step - loss: 31391768.0000 

Epoch 216/500
4/4 [==============================] - 0s 12ms/step - loss: 29775016.0000 - mae: 4217.6807 - val_loss: 19440138.0000 - val_mae: 3324.7583
Epoch 217/500
4/4 [==============================] - 0s 12ms/step - loss: 29804212.0000 - mae: 4177.3833 - val_loss: 20162508.0000 - val_mae: 3475.5017
Epoch 218/500
4/4 [==============================] - 0s 10ms/step - loss: 29798098.0000 - mae: 4299.0210 - val_loss: 19828890.0000 - val_mae: 3407.0518
Epoch 219/500
4/4 [==============================] - 0s 12ms/step - loss: 29673168.0000 - mae: 4278.3364 - val_loss: 19432098.0000 - val_mae: 3332.9683
Epoch 220/500
4/4 [==============================] - 0s 13ms/step - loss: 29590512.0000 - mae: 4200.2212 - val_loss: 19945342.0000 - val_mae: 3441.8215
Epoch 221/500
4/4 [==============================] - 0s 10ms/step - loss: 29897866.0000 - mae: 4327.2944 - val_loss: 19124788.0000 - val_mae: 3282.3669
Epoch 222/500
4/4 [==============================] - 0s 11ms/step - loss: 29708906.0000 

Epoch 323/500
4/4 [==============================] - 0s 11ms/step - loss: 24118360.0000 - mae: 3815.3840 - val_loss: 15570024.0000 - val_mae: 2967.0044
Epoch 324/500
4/4 [==============================] - 0s 11ms/step - loss: 24373044.0000 - mae: 3846.3901 - val_loss: 16159036.0000 - val_mae: 3110.5662
Epoch 325/500
4/4 [==============================] - 0s 9ms/step - loss: 23918324.0000 - mae: 3916.5532 - val_loss: 15268102.0000 - val_mae: 2899.7041
Epoch 326/500
4/4 [==============================] - 0s 12ms/step - loss: 24015568.0000 - mae: 3786.1819 - val_loss: 15164984.0000 - val_mae: 2875.6560
Epoch 327/500
4/4 [==============================] - 0s 11ms/step - loss: 23921912.0000 - mae: 3702.2556 - val_loss: 15434055.0000 - val_mae: 2959.6052
Epoch 328/500
4/4 [==============================] - 0s 13ms/step - loss: 23708728.0000 - mae: 3785.0754 - val_loss: 15434489.0000 - val_mae: 2967.1289
Epoch 329/500
4/4 [==============================] - 0s 11ms/step - loss: 23749118.0000 -

...

Epoch 225/500
4/4 [==============================] - 0s 12ms/step - loss: 23119922.0000 - mae: 3662.4880 - val_loss: 39092112.0000 - val_mae: 4399.2168
Epoch 226/500
4/4 [==============================] - 0s 13ms/step - loss: 23092712.0000 - mae: 3601.4807 - val_loss: 38815108.0000 - val_mae: 4397.0474
Epoch 227/500
4/4 [==============================] - 0s 11ms/step - loss: 23163368.0000 - mae: 3599.2642 - val_loss: 37719732.0000 - val_mae: 4414.6855
Epoch 228/500
4/4 [==============================] - 0s 13ms/step - loss: 23019506.0000 - mae: 3705.9590 - val_loss: 38233328.0000 - val_mae: 4393.7793
Epoch 229/500
4/4 [==============================] - 0s 12ms/step - loss: 23066244.0000 - mae: 3614.0479 - val_loss: 37884976.0000 - val_mae: 4397.1177
Epoch 230/500
4/4 [==============================] - 0s 12ms/step - loss: 22918934.0000 - mae: 3671.7529 - val_loss: 38325636.0000 - val_mae: 4383.8267
Epoch 231/500
4/4 [==============================] - 0s 11ms/step - loss: 22959538.0000 

Epoch 279/500
4/4 [==============================] - 0s 12ms/step - loss: 21040636.0000 - mae: 3470.1458 - val_loss: 33945748.0000 - val_mae: 4218.5259
Epoch 280/500
4/4 [==============================] - 0s 13ms/step - loss: 20960406.0000 - mae: 3554.1162 - val_loss: 35591280.0000 - val_mae: 4153.8672
Epoch 281/500
4/4 [==============================] - 0s 11ms/step - loss: 21180542.0000 - mae: 3439.0249 - val_loss: 36257464.0000 - val_mae: 4144.9233
Epoch 282/500
4/4 [==============================] - 0s 12ms/step - loss: 20917682.0000 - mae: 3382.7942 - val_loss: 36548884.0000 - val_mae: 4138.3745
Epoch 283/500
4/4 [==============================] - 0s 13ms/step - loss: 21064586.0000 - mae: 3337.1423 - val_loss: 34579040.0000 - val_mae: 4146.5361
Epoch 284/500
4/4 [==============================] - 0s 12ms/step - loss: 21312418.0000 - mae: 3505.1123 - val_loss: 35272920.0000 - val_mae: 4137.9224
Epoch 285/500
4/4 [==============================] - 0s 14ms/step - loss: 20786256.0000 

Epoch 333/500
4/4 [==============================] - 0s 13ms/step - loss: 18548274.0000 - mae: 3248.5298 - val_loss: 32133076.0000 - val_mae: 3848.2183
Epoch 334/500
4/4 [==============================] - 0s 12ms/step - loss: 18682646.0000 - mae: 3177.2222 - val_loss: 30884192.0000 - val_mae: 3871.4321
Epoch 335/500
4/4 [==============================] - 0s 10ms/step - loss: 18398510.0000 - mae: 3190.2639 - val_loss: 30786860.0000 - val_mae: 3864.3545
Epoch 336/500
4/4 [==============================] - 0s 13ms/step - loss: 18360122.0000 - mae: 3174.6941 - val_loss: 29536322.0000 - val_mae: 3922.0383
Epoch 337/500
4/4 [==============================] - 0s 10ms/step - loss: 18569680.0000 - mae: 3334.6450 - val_loss: 29990572.0000 - val_mae: 3873.1929
Epoch 338/500
4/4 [==============================] - 0s 14ms/step - loss: 18301274.0000 - mae: 3264.9729 - val_loss: 30882162.0000 - val_mae: 3826.8794
Epoch 339/500
4/4 [==============================] - 0s 11ms/step - loss: 18128292.0000 

Epoch 387/500
4/4 [==============================] - 0s 12ms/step - loss: 14836917.0000 - mae: 2882.2512 - val_loss: 22793714.0000 - val_mae: 3449.3218
Epoch 388/500
4/4 [==============================] - 0s 12ms/step - loss: 14728385.0000 - mae: 2977.7720 - val_loss: 23622688.0000 - val_mae: 3338.0276
Epoch 389/500
4/4 [==============================] - 0s 14ms/step - loss: 14492735.0000 - mae: 2878.6724 - val_loss: 25332022.0000 - val_mae: 3300.3774
Epoch 390/500
4/4 [==============================] - 0s 11ms/step - loss: 14638249.0000 - mae: 2757.1731 - val_loss: 23373342.0000 - val_mae: 3312.7871
Epoch 391/500
4/4 [==============================] - 0s 11ms/step - loss: 14325835.0000 - mae: 2823.7561 - val_loss: 22965610.0000 - val_mae: 3319.8889
Epoch 392/500
4/4 [==============================] - 0s 13ms/step - loss: 14841659.0000 - mae: 2944.5244 - val_loss: 23225492.0000 - val_mae: 3285.1973
Epoch 393/500
4/4 [==============================] - 0s 12ms/step - loss: 14239040.0000 

Epoch 494/500
4/4 [==============================] - 0s 13ms/step - loss: 10737477.0000 - mae: 2322.9771 - val_loss: 16210723.0000 - val_mae: 2611.8962
Epoch 495/500
4/4 [==============================] - 0s 11ms/step - loss: 10796751.0000 - mae: 2256.2517 - val_loss: 15497904.0000 - val_mae: 2589.6313
Epoch 496/500
4/4 [==============================] - 0s 12ms/step - loss: 10728312.0000 - mae: 2275.7158 - val_loss: 15391309.0000 - val_mae: 2593.6907
Epoch 497/500
4/4 [==============================] - 0s 11ms/step - loss: 11203979.0000 - mae: 2394.9392 - val_loss: 15959232.0000 - val_mae: 2593.7354
Epoch 498/500
4/4 [==============================] - 0s 11ms/step - loss: 10911552.0000 - mae: 2292.9109 - val_loss: 15565435.0000 - val_mae: 2582.5637
Epoch 499/500
4/4 [==============================] - 0s 12ms/step - loss: 11053573.0000 - mae: 2319.1736 - val_loss: 14775014.0000 - val_mae: 2657.8086
Epoch 500/500
4/4 [==============================] - 0s 10ms/step - loss: 11036151.0000 

In [22]:
history.history.keys()

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])

In [24]:
test_mse_score, test_mae_score = model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 14581768.0000 - mae: 2562.0300


In [43]:
test_mae_score

2562.030029296875

In [30]:
pred=model.predict(x_test)

In [31]:
pred.reshape(-1)
pred=np.round(pred)
pred

array([[26315.],
       [19993.],
       [10349.],
       [13047.],
       [24380.],
       [ 7113.],
       [ 8544.],
       [ 7747.],
       [10899.],
       [ 7780.],
       [14168.],
       [ 8145.],
       [15153.],
       [10734.],
       [34599.],
       [ 6877.],
       [ 3705.],
       [19461.],
       [10778.],
       [10160.],
       [10210.],
       [22475.],
       [ 7057.],
       [ 5587.],
       [ 6935.],
       [27126.],
       [14587.],
       [16815.],
       [ 7054.],
       [15886.],
       [22591.],
       [ 6807.],
       [ 8195.],
       [19257.],
       [ 9752.],
       [22824.],
       [12222.],
       [ 9486.],
       [ 6538.],
       [20169.],
       [ 9064.],
       [12128.],
       [14633.],
       [ 8154.],
       [ 6788.],
       [ 9542.],
       [ 6807.],
       [ 8331.],
       [17631.],
       [19181.],
       [ 6334.],
       [21289.],
       [ 6859.],
       [10125.],
       [ 6978.],
       [13231.],
       [14618.],
       [ 9486.],
       [25734.

In [32]:
evaluate=pd.DataFrame({'Actual': y_test['price']})
evaluate['prediction']= pred
evaluate

,Actual,prediction
15,30760.000,26315.0
9,17859.167,19993.0
100,9549.000,10349.0
132,11850.000,13047.0
68,28248.000,24380.0
...,...,...
56,11845.000,9486.0
128,37028.000,25734.0
76,5389.000,6548.0
144,9233.000,9713.0
